In [1]:
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("//dbfs/FileStore/tables/nse_full/*.csv")
#problem is we do not know what file dat came from, so this approach does not work for multiple company file merge where stock name is not resent.

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4326794317003050> in <module> () 
 1 df = sqlContext . read . format ( "com.databricks.spark.csv" ) . option ( "header" , "true" ) . load ( "/FileStore/tables/allnsezip/*.csv" ) 
 ----> 2 dfWithCol = df . withColumn ( "filename" , input_file_name ( ) ) 

 NameError : name 'input_file_name' is not defined

In [2]:
df.printSchema() 

root
-- Date: string (nullable = true)
-- open: string (nullable = true)
-- high: string (nullable = true)
-- low: string (nullable = true)
-- close: string (nullable = true)
-- adj_close: string (nullable = true)
-- volume: string (nullable = true)

In [3]:
import pandas as pd
import glob

#globbed_files = glob.glob("/dbfs/FileStore/tables/nserelevant/*.csv") #creates a list of all csv files
globbed_files = glob.glob("/dbfs/FileStore/tables/nse_full/*.csv")
data = [] # pd.concat takes a list of dataframes as an agrument
import os

for csv in globbed_files:
    frame = pd.read_csv(csv)
    companyname =os.path.basename(csv)
    companyname = companyname[:-9]
    frame['ticker'] = companyname
    data.append(frame)

bigframe = pd.concat(data, ignore_index=True) #dont want pandas to try an align row indexes
bigframe=bigframe[bigframe.open.str.contains("null") == False] 
bigframe.to_csv("/dbfs/FileStore/tables/allnsedata/nsecombinedall.csv")

In [4]:
bigframe.count()

Out[ 8 ]: 
Date 3491328
open 3491328
high 3491328
low 3491328
close 3491328
adj_close 3491328
volume 3491328
ticker 3491328
dtype: int64

In [5]:
bigframe.ticker.unique()

Out[ 6 ]: 
array(['20MICRON', '3IINFOTEC', '3MINDI', ..., 'ZODJRDMK', 'ZYDUSWEL',
 'ZYLO'], dtype=object)